<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: To execute the Python code in the code cell below, click on the cell to select it and press <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>

### Required Libraries

In [ ]:
from aeon.segmentation import find_dominant_window_sizes

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
import fnmatch
import zipfile

%config InlineBackend.figure_formats = {'png', 'retina'}

### Utility functions

def read_series(file, locations):
    # file is e.g. "000_Anomaly_2500.csv"
    internal_name = folder_in_zip + file
    print(internal_name)

    with zf.open(internal_name) as f:
        data = pd.read_csv(f, header=None)

    data = np.array(data).flatten()

    # Extract file name
    file_name = file.split('.')[0]
    splits = file_name.split('_')
    test_start = np.array(splits[-1])

    # Extract anomaly location
    anomaly = (-1, -1)
    if file_name in locations.index:
        row = locations.loc[file_name]
        anomaly = row["Start"], row["End"]    

    return (file_name, int(test_start), data, anomaly)

def sliding_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)


def visualize_with_anomaly_score(
        data, score, test_start, 
        predicted, anomaly, name = None
        ):
    '''Input:
       data: array with the raw data
       test_start: the offset where the test data starts
       predicted: The offset of your prediction.
       anomaly: The offset of the anomaly. 
                      If -1 is passed, no anomaly is plottet.
    '''
    
    anomaly_start = anomaly[0]
    anomaly_end = anomaly[1]
    predicted_start = max(0, predicted - 50)
    predicted_end = min(predicted + 50, data.shape[-1])
    
    fig, ax = plt.subplots(2,1, figsize=(20, 4), sharex=True)
    sns.lineplot(x=np.arange(test_start, len(data)), y=data[test_start:], ax = ax[0], lw=0.5, label="Test")
    sns.lineplot(x=np.arange(0, test_start), y=data[:test_start], ax = ax[0], lw=0.5, label="Train")
        
    if (anomaly_start > 0):
        sns.lineplot(x=np.arange(anomaly_start, anomaly_end), 
                     y=data[anomaly_start:anomaly_end], ax = ax[0], label="Actual", color="green")
    
    sns.lineplot(x=np.arange(len(score)), y=score, ax = ax[1], label="Anomaly Scores")
    sns.lineplot(x=np.arange(predicted_start, predicted_end), 
                 y=data[predicted_start:predicted_end], ax = ax[0], label="Predicted", color="red")

    if name is not None:
        ax[0].set_title(name)
        
    sns.despine()

    plt.legend()
    plt.show()


locations = pd.read_csv("labels.csv")
locations.set_index("Name", inplace=True)

zip_path = "phase_1.zip"
folder_in_zip = "phase_1/"

# Pre-open the zip for listing files
zf = zipfile.ZipFile(zip_path)

# All CSV files inside phase_1/ (excluding labels.csv for the widget)
file_list = np.sort([
    name[len(folder_in_zip):]  # strip "phase_1/" so widget shows just the filename
    for name in zf.namelist()
    if name.startswith(folder_in_zip)
       and fnmatch.fnmatch(name, "*.csv")
       and not name.endswith("labels.csv")
])    

# Exercise 3 - Anomaly Detection

## Dataset
In this notebook, you must implement methods for detecting anomalies in time series.

<img src="https://box.hu-berlin.de/f/53a91798173c4dad9345/?dl=1" width=800/>

You are given 30 time series with clean train segments and anomalous test segments:

- TRAIN: Normal data only
- TEST:  Exactly 1 anomaly per series (unknown size/shape)

### TASK: Implement $\geq 3$ anomaly detection approaches, with two coming from different categories.

1. Distance-Based (e.g., k-Nearest Neighbor Classifier)
2. Model-Based (e.g., Isolation Forest, LOF, One-Class-SVM)
3. Regression-Based (e.g., linear / polynomial regression)
4. Forecasting-Based (e.g., Prophet, ARIMA)
5. Clustering-Based (e.g., DBSCAN)
6. Statistics-Based (e.g., IQR, KDE)

**Hints:**
- You may use Vibe Coding to solve this exercise.
- You may play around with **pre-processing**. **This counts as a different approach**

### You must hand in this exercise via Moodle and submit predictions to Kaggle.

<hr/>

# Anomaly Detection using Predictive Modelling

You are given wrappers, which you can use to implement methods. Each wrapper returns a score-profile and the predicted anomaly. The score's maximum indicates where the anomaly is.

<img src="https://box.hu-berlin.de/f/2812ccfbcae24e318f9e/?dl=1" width=800/>

In [ ]:
# Distance-based anomaly scores
# e.g. k-Nearest Neighbor Classifier
# see: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
def nearest_neighbor_based_score(X, test_start):
    score = np.zeros(len(X))
    # TODO: fill in code here as needed 

    from sklearn.neighbors import NearestNeighbors
    # Fit kNN on training data only
    knn = NearestNeighbors(n_neighbors=5)
    knn.fit(X[:test_start - window_size + 1])
    
    # For each point, get distance to nearest neighbors in training set
    distances, _ = knn.kneighbors(X)
    
    # Average distance to k neighbors = anomaly score
    score = np.mean(distances, axis=1)
    
    # Pay attention to using argmin or argmax!
    predicted = test_start + np.argmax(score[test_start:])
    return score, predicted


# Use a classification approach to detect anaomalies
# e.g. e.g., Isolation Forest, LOF, One-Class-SVM
# see: https://scikit-learn.org/stable/modules/outlier_detection.html
def classifcation_based_scores(X, test_start):   
    score = np.zeros(len(X))
    # TODO: fill in code here as needed 

    from sklearn.ensemble import IsolationForest
    from sklearn.svm import OneClassSVM
    from sklearn.neighbors import LocalOutlierFactor
    from sklearn.preprocessing import RobustScaler
    from scipy.ndimage import gaussian_filter1d
    import warnings
    warnings.filterwarnings('ignore')
    
    # === 1. Feature Engineering ===
    features = []
    for i in range(len(X)):
        window = X[i]
        feat = [
            np.mean(window),
            np.std(window),
            window[-1],
            window[-1] - window[0],  # Trend
            np.max(window) - np.min(window),  # Range
            np.percentile(window, 75) - np.percentile(window, 25),  # IQR
        ]
        # First derivative features
        diffs = np.diff(window)
        feat.extend([np.mean(diffs), np.std(diffs), np.max(np.abs(diffs))])
        # Second derivative
        diffs2 = np.diff(diffs)
        if len(diffs2) > 0:
            feat.extend([np.mean(diffs2), np.std(diffs2)])
        else:
            feat.extend([0, 0])
        features.append(feat)
    
    X_features = np.array(features)
    
    # === 2. Scaling ===
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X[:test_start - window_size + 1])
    X_all_scaled = scaler.transform(X_features)
    
    # === 3. Ensemble of Classification-Based Methods ===
    all_scores = []
    
    # Method A: Isolation Forest with different contamination values
    for cont in [0.01, 0.05, 0.1]:
        try:
            iso_forest = IsolationForest(contamination=cont, random_state=42, n_estimators=100)
            iso_forest.fit(X_train_scaled)
            # decision_function returns negative for outliers, we negate to get higher=anomalous
            iso_scores = -iso_forest.decision_function(X_all_scaled)
            all_scores.append(iso_scores)
        except:
            pass
    
    # Method B: One-Class SVM
    for nu in [0.01, 0.05, 0.1]:
        try:
            ocsvm = OneClassSVM(nu=nu, kernel='rbf', gamma='auto')
            ocsvm.fit(X_train_scaled)
            # decision_function returns negative for outliers
            svm_scores = -ocsvm.decision_function(X_all_scaled)
            all_scores.append(svm_scores)
        except:
            pass
    
    # Method C: LOF (Local Outlier Factor) with novelty detection
    for n_neighbors in [10, 20, 30, 50]:
        try:
            lof = LocalOutlierFactor(n_neighbors=n_neighbors, novelty=True, contamination=0.01)
            lof.fit(X_train_scaled)
            lof_scores = -lof.decision_function(X_all_scaled)
            all_scores.append(lof_scores)
        except:
            pass
    
    # === 4. Normalize and combine scores ===
    normalized_scores = []
    for s in all_scores:
        s_norm = (s - np.min(s)) / (np.max(s) - np.min(s) + 1e-10)
        normalized_scores.append(s_norm)
    
    combined = np.mean(normalized_scores, axis=0)
    
    # === 5. Heavy smoothing to find anomaly region ===
    window_size = max(20, X.shape[1])
    score = gaussian_filter1d(combined, sigma=window_size/3)
    
    # Zero out training region
    score[:test_start] = 0
    
    # Pay attention to using argmin or argmax!
    predicted = test_start + np.argmin(score[test_start:])
    return score, predicted


    
# Use a regression-based approach to detect anaomalies
# e.g. linear / polynomial regression
# see: https://scikit-learn.org/stable/modules/linear_model.html
def regression_based_scores(X, test_start):
    score = np.zeros(len(X))
    # TODO: fill in code here as needed 
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.preprocessing import StandardScaler
    from scipy.ndimage import uniform_filter1d
    
    # Preprocessing: Z-score normalize
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(X[:test_start])
    X_all = scaler.transform(X)
    
    # Use multiple lagged values as features
    n_lags = 3
    X_train = X_all[:test_start-n_lags]
    y_train = X_all[n_lags:test_start, -1]
    
    # Use Gradient Boosting for better predictions
    model = GradientBoostingRegressor(
        n_estimators=100,
        max_depth=3,
        learning_rate=0.1,
        random_state=42
    )
    model.fit(X_train, y_train)
    
    # Predict for all data
    y_pred = model.predict(X_all[:-n_lags])
    y_actual = X_all[n_lags:, -1]
    
    # Use absolute residuals (more robust)
    residuals = np.abs(y_actual - y_pred)
    
    # Apply smoothing to reduce noise
    window_smooth = max(5, X.shape[1] // 4)
    residuals_smooth = uniform_filter1d(residuals, size=window_smooth)
    
    score[n_lags:len(residuals_smooth)+n_lags] = residuals_smooth
    
    # Pay attention to using argmin or argmax!
    predicted = test_start + np.argmax(score[test_start:])
    return score, predicted

    
# Use a forecasting-based approach to detect anaomalies
# e.g. ARIMA, Prophet
# see: https://medium.com/@aeon.toolkit/blazingly-fast-arima-with-aeon-9796d7015cdd
def forecasting_based_scores(X, test_start):
    score = np.zeros(len(X))
    # TODO: fill in code here as needed 

    from sklearn.neighbors import LocalOutlierFactor, NearestNeighbors
    from sklearn.linear_model import Ridge
    from sklearn.preprocessing import RobustScaler
    from scipy.ndimage import gaussian_filter1d
    import warnings
    warnings.filterwarnings('ignore')
    
    # === 1. Feature Engineering with temporal features ===
    features = []
    for i in range(len(X)):
        window = X[i]
        feat = [
            np.mean(window),
            np.std(window),
            window[-1],
            window[-1] - window[0],
            np.max(window) - np.min(window),
            np.percentile(window, 75) - np.percentile(window, 25),
        ]
        diffs = np.diff(window)
        feat.extend([np.mean(diffs), np.std(diffs), np.max(np.abs(diffs))])
        
        # Add second derivative
        diffs2 = np.diff(diffs)
        if len(diffs2) > 0:
            feat.extend([np.mean(diffs2), np.std(diffs2)])
        else:
            feat.extend([0, 0])
        
        features.append(feat)
    
    X_features = np.array(features)
    
    # === 2. Scaling ===
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X[:test_start - window_size + 1])
    X_all_scaled = scaler.transform(X_features)
    
    all_scores = []
    
    # === 3. Prediction residuals ensemble ===
    for lag in [1, 2, 3, 5]:
        if test_start > lag + 10:
            X_in = X_train_scaled[:-lag]
            y_out = X_train_scaled[lag:]
            
            model = Ridge(alpha=1.0)
            model.fit(X_in, y_out)
            
            pred_scores = np.zeros(len(X_all_scaled))
            for i in range(len(X_all_scaled) - lag):
                pred = model.predict(X_all_scaled[i:i+1])
                actual = X_all_scaled[i + lag]
                pred_scores[i + lag] = np.linalg.norm(actual - pred)
            
            if np.max(pred_scores) > 0:
                pred_scores = pred_scores / np.max(pred_scores)
            all_scores.append(pred_scores)
    
    # === 4. LOF on temporal context (like cluster method) ===
    for n_neighbors in [10, 20, 30]:
        try:
            lof = LocalOutlierFactor(n_neighbors=n_neighbors, novelty=True, contamination=0.01)
            lof.fit(X_train_scaled)
            lof_scores = -lof.decision_function(X_all_scaled)
            lof_scores = (lof_scores - np.min(lof_scores)) / (np.max(lof_scores) - np.min(lof_scores) + 1e-10)
            all_scores.append(lof_scores)
        except:
            pass
    
    # === 5. kNN distance (key component from cluster method) ===
    for k in [5, 10, 15]:
        knn = NearestNeighbors(n_neighbors=k)
        knn.fit(X_train_scaled)
        distances, _ = knn.kneighbors(X_all_scaled)
        avg_dist = np.mean(distances, axis=1)
        avg_dist = (avg_dist - np.min(avg_dist)) / (np.max(avg_dist) - np.min(avg_dist) + 1e-10)
        all_scores.append(avg_dist)
    
    # === 6. Combine ===
    combined = np.mean(all_scores, axis=0)
    
    # === 7. Heavy smoothing (same as cluster) ===
    window_size = max(20, X.shape[1])
    score = gaussian_filter1d(combined, sigma=window_size/3)
    
    # Zero training
    score[:test_start] = 0
    
    # Pay attention to using argmin or argmax!
    predicted = test_start + np.argmax(score[test_start:])
    return score, predicted


# Use a clustering approach to detect anaomalies
# e.g., DBSCAN
# see: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html
def cluster_based_scores(X, test_start):
    
    import numpy as np
    from sklearn.cluster import KMeans
    from sklearn.neighbors import LocalOutlierFactor, NearestNeighbors
    from sklearn.preprocessing import RobustScaler, MinMaxScaler
    from scipy.ndimage import gaussian_filter1d
    import warnings
    warnings.filterwarnings('ignore')



    # === 1. Vectorized Feature Engineering ===

    f_std = np.std(X, axis=1)
    f_trend = X[:, -1] - X[:, 0]
    f_range = np.ptp(X, axis=1) 
    f_iqr = np.percentile(X, 75, axis=1) - np.percentile(X, 25, axis=1)
    
    diffs = np.diff(X, axis=1)
    f_d_mean = np.mean(diffs, axis=1)
    f_d_std = np.std(diffs, axis=1)
    f_d_max = np.max(np.abs(diffs), axis=1)
    
    # Peak Location
    f_argmax = np.argmax(X, axis=1).astype(float) 
    
    # Assymetrie
    mid_point = X.shape[1] // 2
    left_std = np.std(X[:, :mid_point], axis=1)
    right_std = np.std(X[:, mid_point:], axis=1)
    f_asymmetry = left_std - right_std 

    # Center of Mass
    indices = np.arange(X.shape[1])
    
    X_pos = X - np.min(X, axis=1, keepdims=True) + 1e-5
    f_center_mass = np.sum(indices * X_pos, axis=1) / np.sum(X_pos, axis=1)

    X_features = np.column_stack([
        f_std, f_trend, f_range, f_iqr,
        f_d_mean, f_d_std, f_d_max,
        f_argmax,      
        f_asymmetry,   
        f_center_mass  
    ])

    # === 2. Strict Train/Test Split (No Leakage) ===

    window_size = X.shape[1]
    train_end = max(0, test_start - window_size + 1)

    # RobustScaler 

    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_features[:train_end])
    X_all_scaled = scaler.transform(X_features)

    # === 3. Ensemble ===

    all_scores = []

    # Method A: LOF
    
    for n in [10, 30]: 
        lof = LocalOutlierFactor(n_neighbors=n, novelty=True, contamination=0.001)
        lof.fit(X_train_scaled)
        all_scores.append(-lof.decision_function(X_all_scaled))

    # Method B: KMeans Distance

    for n in [5, 10]: 
        kmeans = KMeans(n_clusters=n, random_state=42, n_init=10)
        kmeans.fit(X_train_scaled)
        distances = kmeans.transform(X_all_scaled)
        all_scores.append(np.min(distances, axis=1))

    # Method C: kNN Distance

    for k in [5, 15]:
        knn = NearestNeighbors(n_neighbors=k)
        knn.fit(X_train_scaled)
        dists, _ = knn.kneighbors(X_all_scaled)
        all_scores.append(np.mean(dists, axis=1))

    # === 4. Robust Combination ===

    normalized_scores = []
    score_scaler = MinMaxScaler() # Можно заменить на RobustScaler если много шума

    for s in all_scores:

        cap = np.percentile(s, 99) 
        s_clipped = np.clip(s, a_min=None, a_max=cap)
        s_norm = score_scaler.fit_transform(s_clipped.reshape(-1, 1)).flatten()
        normalized_scores.append(s_norm)

    combined = np.mean(normalized_scores, axis=0) # Можно попробовать np.max

    # === 5. Smoothing ===

    filter_sigma = max(5, window_size / 3) 
    score = gaussian_filter1d(combined, sigma=filter_sigma)

    # Zero out training region

    score[:test_start] = 0
    
    predicted = test_start + np.argmax(score[test_start:])
    return score, predicted


# Use a statistics approach to detect anaomalies
# e.g., KDE, IQR
def statistics_based_scores(data, X, test_start):   
    score = np.zeros(len(X))
    # TODO: fill in code here as needed 

    from scipy.stats import gaussian_kde, zscore
    from scipy.ndimage import gaussian_filter1d
    from sklearn.preprocessing import RobustScaler
    import warnings
    warnings.filterwarnings('ignore')
    
    # === 1. Feature Engineering ===
    features = []
    for i in range(len(X)):
        window = X[i]
        feat = [
            np.mean(window),
            np.std(window),
            window[-1],
            window[-1] - window[0],
            np.max(window) - np.min(window),
            np.percentile(window, 75) - np.percentile(window, 25),
        ]
        diffs = np.diff(window)
        feat.extend([np.mean(diffs), np.std(diffs), np.max(np.abs(diffs))])
        features.append(feat)
    
    X_features = np.array(features)
    
    # === 2. Scale based on training data ===
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X[:test_start - window_size + 1])
    X_all_scaled = scaler.transform(X_features)
    
    all_scores = []
    
    # === 3. Multi-dimensional KDE ===
    try:
        kde = gaussian_kde(X_train_scaled.T)
        likelihoods = kde(X_all_scaled.T)
        kde_score = 1 / (likelihoods + 1e-10)
        kde_score = (kde_score - np.min(kde_score)) / (np.max(kde_score) - np.min(kde_score) + 1e-10)
        all_scores.append(kde_score)
    except:
        pass
    
    # === 4. Mahalanobis-like distance ===
    train_mean = np.mean(X_train_scaled, axis=0)
    train_cov = np.cov(X_train_scaled.T)
    try:
        inv_cov = np.linalg.inv(train_cov + np.eye(train_cov.shape[0]) * 1e-6)
        mahal_scores = []
        for i in range(len(X_all_scaled)):
            diff = X_all_scaled[i] - train_mean
            mahal = np.sqrt(diff @ inv_cov @ diff.T)
            mahal_scores.append(mahal)
        mahal_scores = np.array(mahal_scores)
        mahal_scores = (mahal_scores - np.min(mahal_scores)) / (np.max(mahal_scores) - np.min(mahal_scores) + 1e-10)
        all_scores.append(mahal_scores)
    except:
        pass
    
    # === 5. Per-feature Z-score outlier detection ===
    z_combined = np.zeros(len(X_all_scaled))
    for col in range(X_all_scaled.shape[1]):
        train_mean_col = np.mean(X_train_scaled[:, col])
        train_std_col = np.std(X_train_scaled[:, col]) + 1e-10
        z_scores = np.abs((X_all_scaled[:, col] - train_mean_col) / train_std_col)
        z_combined += z_scores
    z_combined = (z_combined - np.min(z_combined)) / (np.max(z_combined) - np.min(z_combined) + 1e-10)
    all_scores.append(z_combined)
    
    # === 6. IQR-based multi-feature ===
    iqr_combined = np.zeros(len(X_all_scaled))
    for col in range(X_train_scaled.shape[1]):
        Q1 = np.percentile(X_train_scaled[:, col], 25)
        Q3 = np.percentile(X_train_scaled[:, col], 75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        for i in range(len(X_all_scaled)):
            if X_all_scaled[i, col] < lower:
                iqr_combined[i] += lower - X_all_scaled[i, col]
            elif X_all_scaled[i, col] > upper:
                iqr_combined[i] += X_all_scaled[i, col] - upper
    if np.max(iqr_combined) > 0:
        iqr_combined = iqr_combined / np.max(iqr_combined)
    all_scores.append(iqr_combined)
    
    # === 7. Combine all scores ===
    combined = np.mean(all_scores, axis=0)
    
    # === 8. Heavy smoothing ===
    window_size = max(20, X.shape[1])
    score = gaussian_filter1d(combined, sigma=window_size/3)
    
    # Zero training region
    score[:test_start] = 0
    
    # Pay attention to using argmin or argmax!
    predicted = test_start + np.argmax(score[test_start:])
    return score, predicted



In [ ]:
total_score = 0
predictions = []
ids = []
i = 1

for file in file_list:
    if "Anomaly" in str(file):
        file_name = file.split('.')[0]
        name, test_start, data, anomaly = read_series(file, locations)

        period = find_dominant_window_sizes(data[:test_start])
        print("Dominant Period", period)
        
        window_size = np.int32(period)
        X = sliding_window(data, window_size)

        #score, predicted = nearest_neighbor_based_score(X, test_start)            

        # TODO: uncomment as needed
        #score, predicted = nearest_neighbor_based_score(X, test_start) #SCORES 4
        #score, predicted = classifcation_based_scores(X, test_start) #
        #score, predicted = regression_based_scores(X, test_start) #SCORES 10      
        #score, predicted = forecasting_based_scores(X, test_start) #SCORES 6 (5.5 on Kaggle)
        score, predicted = cluster_based_scores(X, test_start) #OUR BEST WARRIOR
        #score, predicted = statistics_based_scores(data, X, test_start) #SCORES 9.7
        
        predictions.append(predicted)
        ids.append(file_name)        
        score[:test_start] = np.nan

        # Visualize the predicted anomaly
        visualize_with_anomaly_score(
             data, score, test_start, predicted, anomaly, name)
        
        # Only check those, where the anomaly was annotated
        if (anomaly[0] > -1):
            anomaly = (anomaly[0] + anomaly[1]) // 2
            total_score += abs(anomaly - predicted) / (anomaly)
            i = i + 1
            
        print("Current Score: ", (total_score / i) * 100)

        
print("\tTotal score:", (total_score / len(locations)) * 100)

# Submit your solution to Kaggle

You must make at least **3 different submissions to Kaggle**.

<div class="alert alert-success alertsuccess" style="margin-top: 20px">
Create a submission named `submission.csv` using your model and upload it to kaggle:

- https://www.kaggle.com/competitions/time-series-anomaly-detection-exercise-WS-25-26


</div>

In [ ]:
# Create a submission
submission = pd.DataFrame({'ID': ids,'PREDICTED': predictions})

# Visualize the first 5 rows
display(submission)

filename = 'baseline_submission_phase_1.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

<hr> 

# Submit via Moodle:
- Your notebook with three approaches (or three notebooks)
- A html export of this notebook (or the three notebooks)
- The three submissions, which you used for kaggle

### You must hand in this exercise via moodle